In [ ]:
import numpy as np
import colorcet as cc

from bokeh.io import output_file, output_notebook, show
from bokeh.models import LinearColorMapper, WheelZoomTool, ColorBar, Title, Slider, CustomJS
from bokeh.plotting import figure, ColumnDataSource
from bokeh.layouts import column

from vis_tools import read_hysplit_netcdf, grab_gshhg_features 

#############################################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '/opt/hysplit/18040212_taupo_15.0_0.01.nc'

# SPECIFY:
ASH_MIN = 10**-2  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
RES = 'f'  # detail of GMT features ('c', 'l', 'i', 'h', 'f')
#############################################################

model = read_hysplit_netcdf(FILENAME)

src_lon = model.attrs['volcano_location'][1]
src_lat = model.attrs['volcano_location'][0]
lon = model['lon'].values
lat = model['lat'].values

td = model['total_deposition'].values      
td = np.log10(td)  # manually take the log of the ashfall thickness values

# transform 3-D array into a list of matrices for Bokeh
stack = []
for i in range(len(model['time'])):
    stack.append(td[:,:,i])

features = grab_gshhg_features(RES, [1, 2, 3, 4], [166, 180, -48, -34])  # this includes all of NZ

# create Bokeh figure
p = figure(tools='pan, reset')
wz = WheelZoomTool(zoom_on_axis=False)  # restrict zooming behavior
p.add_tools(wz)
p.toolbar.active_scroll = wz

# PLOT model
slice_src = ColumnDataSource(data=dict(x=[np.min(lon)], y=[np.min(lat)], dw=[np.max(lon)-np.min(lon)],\
                                       dh=[np.max(lat)-np.min(lat)], slice=[stack[0]])
                            )
stack_src = ColumnDataSource(data=dict(stack=stack))

cmapper = LinearColorMapper(palette=cc.fire[::-1], low=np.log10(ASH_MIN), high=np.log10(ASH_MAX), nan_color=(0,0,0,0))
p.image('slice', x='x', y='y', dw='dw', dh='dh', color_mapper=cmapper, source=slice_src)

# PLOT features
p.multi_line(features['longitude'], features['latitude'], color='black')

# PLOT source location
p.scatter(src_lon, src_lat, size=15, marker='triangle', line_color='black', fill_color='cyan', legend='source')

# add colorbar
color_bar = ColorBar(color_mapper=cmapper, location=(0,0), orientation='horizontal')
color_bar.title = 'log\N{SUBSCRIPT ONE}\N{SUBSCRIPT ZERO} [ ash thickness (mm) ]'
p.add_layout(color_bar, 'below')

p.xaxis.axis_label = 'lon'
p.yaxis.axis_label = 'lat'
p.add_layout(Title(text=FILENAME.split('/')[-1]), 'above')
p.add_layout(Title(text='UNPROJECTED'), 'above')

p.match_aspect = True  # plot degree intervals as equal distances (DISTORTED! BAD! UGLY! BUT ACCURATE!)
p.xgrid.visible = False
p.ygrid.visible = False

# time slider callback function
callback = CustomJS(args=dict(slice_src=slice_src, stack_src=stack_src), code='''
    var stack_src = stack_src.data;
    var i = Math.round(hrs.value/3);
    
    slice_src.data['slice'] = [stack_src['stack'][i]];    
    slice_src.change.emit();
''')
        
slider = Slider(start=0, end=24, value=0, step=3, title='hours after eruption onset', callback=callback)
callback.args['hrs'] = slider
    
interface = column(slider, p)

# UNCOMMENT the line below to show output in the notebook
output_notebook(hide_banner=True); show(interface)

# UNCOMMENT the line below to save the output to a standalone HTML file
#output_file('/opt/shared_nbs/ashfall_visual/bokeh_html/bokeh_map.html'); show(interface); print('HTML file saved')